Brick CNot with random single Qubit gates

In [15]:

# import numpy as np
from qiskit import QuantumCircuit, transpile
# from qiskit.quantum_info import Kraus, SuperOp
from qiskit_aer import AerSimulator
from qiskit.tools.visualization import plot_histogram
from qiskit.quantum_info import DensityMatrix, partial_trace
import random
import numpy as np
import math

Brick cnot clifford

In [72]:

def random_clifford(n,qc):
    qubits_list=list(range(n))
    for i in range(500):
        choice=random.choices(['s','h','cnot'])[0]
        qbit=random.choices(qubits_list)[0]
        if choice=='s':
            qc.s(qbit)
        elif choice=='h':
            qc.h(qbit)
        elif choice=='cnot':
            c=qubits_list.copy()
            c.remove(qbit)
            control_bit=random.choices(c)[0]
            qc.cnot(control_bit,qbit)
    return qc
def Block_Cnot_clifford(n,qc,first):
    if(first==True):
        depth=500
    else:
        depth=math.log(n,2)
    layer=0

    while(depth>layer):
        ##layer of even random s and t gate
        for i in range(n):
            choice=random.choices(['s','h'])[0]
            if(choice=='s'):
                qc.s(i)
            elif(choice=='h'):
                qc.h(i)
        layer+=1
        ##layer of even cnot gate 
        for i in range(0,n-1,2):
            qc.cnot(i,i+1)
        layer+=1
        
        ##layer of odd random s and t gate again 
        for i in range(1,n-1,1):
            choice=random.choices(['s','h'])[0]
            if(choice=='s'):
                qc.s(i)
            elif(choice=='h'):
                qc.h(i)
        layer+=1
        
        ##layer of odd cnot    
        for i in range(1,n-2,2):
            qc.cnot(i,i+1)
        layer+=1
    return qc

In [86]:
n=8
qc=QuantumCircuit(n)
##initialization
for i in range(n):
    qc.initialize([1,0],i)
# qc=Block_Cnot_clifford(n,qc,first=True)

qc=random_clifford(n,qc)
choice=random.choices(list(range(n)))[0]
qc.t(choice)
qc=random_clifford(n,qc)

# qc=Block_Cnot_clifford(n,qc,first=False)
# qc.measure_all()


####creating bell state to verify if the density matrix and the partial density matrix i calculates is correct or not
# qc.h(0)
# qc.cnot(0,1)
qc.draw()

┌─────────────────┐┌───┐          ┌───┐┌───┐               ┌───┐┌───┐┌───┐»
q_0: ┤ Initialize(1,0) ├┤ S ├───────■──┤ H ├┤ S ├────────────■──┤ S ├┤ S ├┤ S ├»
     ├─────────────────┤├───┤┌───┐  │  ├───┤└───┘            │  └───┘├───┤├───┤»
q_1: ┤ Initialize(1,0) ├┤ S ├┤ H ├──┼──┤ H ├─────────────────┼────■──┤ H ├┤ S ├»
     ├─────────────────┤├───┤└───┘  │  ├───┤     ┌───┐     ┌─┴─┐  │  ├───┤└───┘»
q_2: ┤ Initialize(1,0) ├┤ H ├──■────┼──┤ X ├─────┤ X ├──■──┤ X ├──┼──┤ S ├─────»
     ├─────────────────┤├───┤  │  ┌─┴─┐└─┬─┘┌───┐└─┬─┘  │  └───┘  │  ├───┤┌───┐»
q_3: ┤ Initialize(1,0) ├┤ H ├──┼──┤ X ├──┼──┤ H ├──┼────┼────■────┼──┤ S ├┤ S ├»
     ├─────────────────┤├───┤  │  └───┘  │  └───┘  │  ┌─┴─┐  │  ┌─┴─┐└───┘└───┘»
q_4: ┤ Initialize(1,0) ├┤ H ├──┼─────────┼─────────┼──┤ X ├──┼──┤ X ├──────────»
     ├─────────────────┤└───┘  │         │  ┌───┐  │  ├───┤  │  ├───┤          »
q_5: ┤ Initialize(1,0) ├───────┼─────────■──┤ S ├──┼──┤ H ├──┼──┤ H ├──────────»
     ├─────────────────┤       │            └───┘  │  └───┘  │  └───┘          »
q_6: ┤ Initialize(1,0) ├───────┼───────────────────┼─────────┼─────────────────»
     ├─────────────────┤┌───┐┌─┴─┐                 │       ┌─┴─┐┌───┐          »
q_7: ┤ Initialize(1,0) ├┤ H ├┤ X ├─────────────────■───────┤ X ├┤ S ├──────────»
     └─────────────────┘└───┘└───┘                         └───┘└───┘          »
«                              ┌───┐┌───┐┌───┐┌───┐┌───┐                    »
«q_0: ──────────────────────■──┤ X ├┤ S ├┤ H ├┤ S ├┤ S ├────────────────────»
«                           │  └─┬─┘├───┤├───┤└───┘├───┤┌───┐               »
«q_1: ──────────────────────┼────■──┤ S ├┤ H ├─────┤ X ├┤ S ├───────────────»
«               ┌───┐┌───┐  │  ┌───┐├───┤└───┘     └─┬─┘└───┘     ┌───┐┌───┐»
«q_2: ──■───────┤ X ├┤ H ├──┼──┤ S ├┤ H ├───────■────┼─────────■──┤ S ├┤ H ├»
«       │  ┌───┐└─┬─┘└───┘  │  ├───┤├───┤┌───┐┌─┴─┐  │  ┌───┐  │  ├───┤├───┤»
«q_3: ──┼──┤ S ├──┼─────────┼──┤ X ├┤ S ├┤ H ├┤ X ├──┼──┤ H ├──┼──┤ H ├┤ H ├»
«       │  ├───┤  │  ┌───┐  │  └─┬─┘├───┤├───┤└───┘  │  ├───┤  │  ├───┤├───┤»
«q_4: ──┼──┤ X ├──┼──┤ S ├──┼────■──┤ S ├┤ H ├──■────┼──┤ X ├──┼──┤ H ├┤ S ├»
«       │  └─┬─┘  │  └───┘  │       └───┘└───┘  │    │  └─┬─┘  │  └───┘└───┘»
«q_5: ──┼────┼────┼────■────┼───────────────────┼────┼────■────┼────────────»
«     ┌─┴─┐  │    │    │  ┌─┴─┐┌───┐          ┌─┴─┐  │  ┌───┐  │  ┌───┐┌───┐»
«q_6: ┤ X ├──■────┼────┼──┤ X ├┤ S ├──────────┤ X ├──┼──┤ H ├──┼──┤ S ├┤ S ├»
«     └───┘       │  ┌─┴─┐└───┘└───┘          └───┘  │  └───┘┌─┴─┐└───┘└───┘»
«q_7: ────────────■──┤ X ├───────────────────────────■───────┤ X ├──────────»
«                    └───┘                                   └───┘          »
«                              ┌───┐┌───┐┌───┐               ┌───┐          »
«q_0: ────────────■────────────┤ X ├┤ H ├┤ H ├────────────■──┤ S ├──────────»
«                 │            └─┬─┘├───┤├───┤┌───┐┌───┐  │  └───┘          »
«q_1: ────────────┼─────────■────┼──┤ X ├┤ H ├┤ H ├┤ H ├──┼─────────────────»
«     ┌───┐┌───┐  │  ┌───┐  │    │  └─┬─┘└───┘├───┤├───┤  │  ┌───┐     ┌───┐»
«q_2: ┤ S ├┤ S ├──┼──┤ S ├──┼────┼────┼────■──┤ H ├┤ H ├──┼──┤ H ├─────┤ X ├»
«     ├───┤├───┤┌─┴─┐└───┘  │    │    │    │  └───┘└───┘  │  └───┘     └─┬─┘»
«q_3: ┤ S ├┤ X ├┤ X ├───────┼────┼────┼────┼──────────────┼─────────■────┼──»
«     └───┘└─┬─┘└───┘     ┌─┴─┐  │    │    │  ┌───┐┌───┐  │         │    │  »
«q_4: ───────■────────────┤ X ├──┼────■────┼──┤ S ├┤ H ├──┼─────────┼────┼──»
«                         └───┘  │         │  └───┘└───┘  │         │    │  »
«q_5: ───────────────────────────■─────────┼──────────────┼─────────┼────■──»
«     ┌───┐                                │            ┌─┴─┐┌───┐  │  ┌───┐»
«q_6: ┤ H ├────────────────────────────────┼────────────┤ X ├┤ H ├──┼──┤ S ├»
«     └───┘                              ┌─┴─┐┌───┐┌───┐├───┤└───┘┌─┴─┐├───┤»
«q_7: ───────────────────────────────────┤ X ├┤ S ├┤ H ├┤ S ├─────┤ X ├┤ S ├»
«                                        └───┘└───┘└──

In [87]:
rho=DensityMatrix(qc) ## rho is the density matrix of the whole system
rho_a=partial_trace(state=rho,qargs=list(range(int(n/2)))) ##rho_a is the reduced density matrix for the first a subsystem A(as the paper has divided the whole system into A and B subsystem)
ES=np.linalg.eigvals(rho_a) ##entanglement spectrum is the set of the eigen values of the reduced density matrix
ES.sort()
lambd=np.sqrt(ES)
print("density matrix=",rho)
print("reduced density matrix=",rho_a)
print("entanglement spectrum",ES)
# print(rho_a)
print(sum(lambd))

density matrix= DensityMatrix([[ 0.00666839-1.30104261e-18j, -0.00276214-6.50521303e-19j,
                -0.00276214-6.50521303e-19j, ...,
                 0.00276214+6.50521303e-19j, -0.00276214-6.50521303e-19j,
                -0.00666839+1.30104261e-18j],
               [-0.00276214-3.68628739e-18j,  0.00114411-6.50521303e-19j,
                 0.00114411-6.50521303e-19j, ...,
                -0.00114411+6.50521303e-19j,  0.00114411-6.50521303e-19j,
                 0.00276214+3.68628739e-18j],
               [-0.00276214-3.68628739e-18j,  0.00114411-6.50521303e-19j,
                 0.00114411-6.50521303e-19j, ...,
                -0.00114411+6.50521303e-19j,  0.00114411-6.50521303e-19j,
                 0.00276214+3.68628739e-18j],
               ...,
               [ 0.00276214+3.68628739e-18j, -0.00114411+6.50521303e-19j,
                -0.00114411+6.50521303e-19j, ...,
                 0.00114411-6.50521303e-19j, -0.00114411+6.50521303e-19j,
                -0.00276214-3.6862

## now lets start creating r^telda

In [75]:
print(lambd)
r_k_tilda=[]
for i in range(1,len(lambd)-1):
    del_k=lambd[i-1]-lambd[i]
    del_k_P1=lambd[i]-lambd[i+1]
    r_k_tilda.append(min(del_k,del_k_P1)/max(del_k,del_k_P1))
print(r_k_tilda)
r_mean=np.mean(r_k_tilda)
print(r_mean)

[0.35355339+0.00000000e+00j 0.        +1.93351279e-17j
 0.35355339+0.00000000e+00j 0.        +1.93351279e-17j
 0.35355339+0.00000000e+00j 0.        +1.93351279e-17j
 0.35355339+0.00000000e+00j 0.        +1.93351279e-17j
 0.35355339+0.00000000e+00j 0.        +1.93351279e-17j
 0.35355339+0.00000000e+00j 0.        +1.93351279e-17j
 0.35355339+0.00000000e+00j 0.        +1.93351279e-17j
 0.35355339+0.00000000e+00j 0.        +1.93351279e-17j]
[(-1+0j), (-1+0j), (-1+0j), (-1+0j), (-1+0j), (-1+0j), (-1+0j), (-1+0j), (-1+0j), (-1+0j), (-1+0j), (-1+0j), (-1+0j), (-1+0j)]
(-1+0j)


In [56]:
sim_ideal = AerSimulator()
result_ideal = sim_ideal.run(qc).result()
plot_histogram(result_ideal.get_counts(0))

QiskitError: 'No counts for experiment "0"'